In [1]:
import os
import torch
import numpy as np
from eagle.model.ea_model import EaModel
from fastchat.model import get_conversation_template
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch.nn.functional as F
import numpy as np

os.environ["CUDA_VISIBLE_DEVICES"] = "1"

In [2]:
# Load your model
depth = 7
base_model_path = "/mtc/models/qwen3-8b"
EAGLE_model_path = "/mtc/chenjunyi1/project/SpecForge/outputs/qwen3-8b-eagle3-dynamic-sharegpt/epoch_9_step_603370"
model = EaModel.from_pretrained(
    base_model_path=base_model_path,
    ea_model_path=EAGLE_model_path,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True,
    device_map="auto",
    total_token=depth+1,
    depth=depth,
    top_k=1,
)
model.eval()

# base_model = AutoModelForCausalLM.from_pretrained(
#     base_model_path,
#     torch_dtype=torch.float16,
#     device_map="auto",
# )

# tokenizer = AutoTokenizer.from_pretrained(base_model_path, use_fast=False)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

{'d2t': tensor([     0,      0,      0,  ..., 119648, 119669, 119669]), 'idk_index': tensor(32000), 'fc.weight': tensor([[-0.0703,  0.1602,  0.0400,  ...,  0.0061, -0.0762,  0.0057],
        [-0.0200,  0.0718, -0.0206,  ..., -0.0269,  0.0596,  0.0217],
        [-0.0272,  0.0645, -0.0427,  ..., -0.0732,  0.0135,  0.1069],
        ...,
        [ 0.0302, -0.0996,  0.1021,  ..., -0.0630, -0.0047, -0.0284],
        [-0.0271, -0.0349, -0.0216,  ...,  0.0430, -0.0139,  0.0417],
        [-0.0562,  0.0776,  0.0347,  ...,  0.0035,  0.0315, -0.0374]],
       dtype=torch.bfloat16), 'lm_head.weight': tensor([[-0.0300, -0.0064, -0.0239,  ...,  0.0356, -0.0059, -0.0008],
        [ 0.0272, -0.0025,  0.0032,  ...,  0.0192, -0.0209,  0.0195],
        [ 0.0493,  0.0117,  0.0210,  ...,  0.0199,  0.0459,  0.0427],
        ...,
        [-0.0256,  0.0466, -0.0148,  ..., -0.0155,  0.0586,  0.0187],
        [-0.0466,  0.0077,  0.0177,  ...,  0.0101,  0.0371, -0.0090],
        [ 0.0013,  0.0225,  0.0103,  ..., 

KeyError: 'd2t.weight'

In [13]:
prompt = "Complete the code I provided.from typing import List\n\n\ndef filter_by_substring(strings: List[str], substring: str) -> List[str]:\n    \"\"\" Filter an input list of strings only for ones that contain given substring\n    >>> filter_by_substring([], 'a')\n    []\n    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')\n    ['abc', 'bacd', 'array']\n    \"\"\"\n"
messages = [
    {"role": "user", "content": prompt}
]
# Note: this draft model is used for thinking mode disabled
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False
)
model_inputs = tokenizer([text], return_tensors="pt").to(base_model.device)

input_ids = model_inputs["input_ids"]
prompt_len = input_ids.shape[1]

max_new_tokens = 2048

generated_ids = input_ids.clone()
top1_probs = []     # p(x_t | x_<t>)
tokens = []
total_steps = 0

for _ in range(max_new_tokens):
    out = base_model(input_ids=generated_ids)
    logits = out.logits[:, -1, :]           # [1, vocab]
    next_token = logits.argmax(dim=-1)      # [1, ]
    top1_probs.append(F.softmax(logits, dim=-1)[0, next_token].item())
    tokens.append(next_token.item())

    generated_ids = torch.cat(
        [generated_ids, next_token[:, None]], dim=1
    )
    total_steps += 1
    if next_token.item() == tokenizer.eos_token_id:
        break

print(tokenizer.decode(generated_ids[0], skip_special_tokens=True))
print(len(generated_ids[0]), total_steps)

user
Complete the code I provided.from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """

assistant
<think>

</think>

Here's the completed code for your function:

```python
from typing import List

def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """
    return [s for s in strings if substring in s]
```

### Explanation:
- The function uses a list comprehension to filter the input list `strings`.
- It includes only those strings that contain the `substring` as a part 

In [14]:
import copy
import json
import time

import torch
import torch.nn as nn
from huggingface_hub import hf_hub_download
from transformers import AutoTokenizer
import os
from transformers import PreTrainedModel, PretrainedConfig, AutoConfig

from eagle.model.modeling_llama_kv import LlamaForCausalLM as KVLlamaForCausalLM
from eagle.model.modeling_mixtral_kv import MixtralForCausalLM as KVMixtralForCausalLM
#from .modeling_qwen2_kv import LlamaForCausalLM as KVQwen2ForCausalLM
from eagle.model.modeling_qwen2_kv import Qwen2ForCausalLM as KVQwen2ForCausalLM
from eagle.model.modeling_qwen3_kv import Qwen3ForCausalLM as KVQwen3ForCausalLM
from eagle.model.utils import *
from eagle.model.kv_cache import initialize_past_key_values

from eagle.model.cnets import Model
from eagle.model.cnets1 import Model as Model1
from eagle.model.configs import EConfig

def reject_all_exit_hook(hook_state):
    return 0

def new_evaluate_posterior(
        logits: torch.Tensor,
        candidates: torch.Tensor,
        logits_processor,
):
    accept_length = 0
    best_candidate = torch.tensor(0, dtype=torch.long, device=candidates.device)
    return best_candidate, accept_length, logits[best_candidate, accept_length]

def new_eagenerate_with_early_exit_hook(
    self,
    input_ids,
    temperature=0.0,
    top_p=0.0,
    top_k=1,            # by default, use greedy sampling for draft model, and only verify draft token chain
    max_new_tokens=512,
    max_length=2048,
    log=False,
    is_llama3=False,
    early_exit_hook=None,  # Hook function to predict accept length for early exit
):
    """
    Generate tokens with early exit hook for MTP module evaluation.
    
    Args:
        early_exit_hook: A callable that takes (logits, candidates, hidden_state_new, outputs, step) 
                        and returns predicted accept length for early exit simulation.
    
    Returns:
        If log=False: input_ids
        If log=True: (input_ids, stats_dict) where stats_dict contains:
            - total_steps: total number of decoding steps
            - total_tokens: total number of generated tokens
            - true_accept_lengths: list of true accept lengths at each step
            - predicted_accept_lengths: list of predicted accept lengths at each step
            - early_exit_triggered: list of bools indicating if early exit was triggered
            - true_acceptance_rate: average true acceptance rate
            - predicted_acceptance_rate: average predicted acceptance rate
            - acceptance_rate_gap: difference between predicted and true rates
            - avg_true_accept_length: average true accept length
            - avg_predicted_accept_length: average predicted accept length
    """
    if is_llama3:
        stop_token_id = self.tokenizer.convert_tokens_to_ids("<|eot_id|>")

    if temperature > 1e-5:
        logits_processor = prepare_logits_processor(temperature=temperature, top_p=top_p, top_k=top_k)
    else:
        logits_processor = None
    
    # Statistics tracking
    stats = {
        'true_accept_lengths': [],
        'predicted_accept_lengths': [],
        'early_exit_triggered': [],
        'total_steps': 0,
        'total_tokens': 0,
    }

    padding = (torch.zeros(1, 1, dtype=torch.long) - 1).to(input_ids.device)
    input_ids = input_ids.clone()
    self.ea_layer.reset_kv()

    # Initialize the past key and value states
    if hasattr(self, "past_key_values"):
        past_key_values = self.past_key_values
        past_key_values_data = self.past_key_values_data
        current_length_data = self.current_length_data
        # Reset the past key and value states
        current_length_data.zero_()
    else:
        (
            past_key_values,
            past_key_values_data,
            current_length_data,
        ) = initialize_past_key_values(self.base_model, max_length=max_length)
        self.past_key_values = past_key_values
        self.past_key_values_data = past_key_values_data
        self.current_length_data = current_length_data

    input_len = input_ids.shape[1]
    reset_tree_mode(self)
    # prefill
    draft_tokens, retrieve_indices, tree_mask, tree_position_ids, logits, hidden_state, sample_token, mtp_logits = initialize_tree(
        input_ids, self, past_key_values, logits_processor
    )
    new_token = 0
    max_length = max_length - self.ea_layer.total_tokens - 10
    sample_p = logits[-1].softmax(dim=-1)

    for idx in range(max_length):
        self.base_model.model.tree_mask = tree_mask

        draft_tokens = draft_tokens.to(input_ids.device)
        # Target model forward, get logits
        logits, hidden_state_new, outputs = tree_decoding(
            self,
            draft_tokens,
            past_key_values,
            tree_position_ids,
            input_ids,
            retrieve_indices,
        )
        
        draft_tokens = torch.cat((draft_tokens, padding), dim=1)
        candidates = draft_tokens[0, retrieve_indices]
        
        # Apply early exit hook BEFORE evaluate_posterior to predict this round's accept length
        # using state information from tree_decoding (logits, hidden_state_new, outputs)
        predicted_accept_length = 0
        early_exit = False
        
        if early_exit_hook is not None:
            # try:
                # Hook gets state information BEFORE verification and predicts accept length
            hook_state = {
                'mtp_module_logits': mtp_logits,
                'draft_candidates': candidates,
                'main_model_logits': logits,
                'main_model_hidden_state': hidden_state_new,
                'main_model_outputs': outputs,
                'step': idx,
                'input_ids': input_ids,
                'retrieve_indices': retrieve_indices,
                'p_token': sample_p,
            }
            predicted_accept_length = early_exit_hook(hook_state)
            
            # Ensure predicted length is valid
            if isinstance(predicted_accept_length, torch.Tensor):
                predicted_accept_length = predicted_accept_length.item()
            predicted_accept_length = max(0, min(predicted_accept_length, candidates.shape[1] - 1))
                
            # except Exception as e:
            #     print(f"Warning: early_exit_hook failed at step {idx}: {e}")
            #     print(hook_state['mtp_module_logits'].shape, 
            #         hook_state['draft_candidates'].shape, 
            #         hook_state['mtp_module_hidden_state'].shape,
            #         hook_state['retrieve_indices'],
            #         hook_state['p_token'].shape,
            #         hook_state['input_ids'].shape)
            #     predicted_accept_length = 0
        
        stats['predicted_accept_lengths'].append(predicted_accept_length)
        
        # Verification - get true accept length
        _, true_accept_length, _ = evaluate_posterior(
            logits, candidates, logits_processor
        )
        best_candidate, accept_length, sample_p = new_evaluate_posterior(
            logits, candidates, logits_processor
        )
        
        # Record true accept length
        # true_accept_length = accept_length.item() if isinstance(accept_length, torch.Tensor) else accept_length
        stats['true_accept_lengths'].append(true_accept_length)
        
        # Check if early exit would trigger (predicted < true)
        if predicted_accept_length < true_accept_length:
            early_exit = True
        
        stats['early_exit_triggered'].append(early_exit)
        
        # Use true accept length for actual generation (to maintain correctness)
        # but record the difference for evaluation
        before_input = input_ids.clone()
        input_ids, draft_tokens, retrieve_indices, tree_mask, tree_position_ids, new_token, hidden_state, sample_token, mtp_logits = update_inference_inputs(
            input_ids,
            candidates,
            best_candidate,
            predicted_accept_length,
            retrieve_indices,
            logits_processor,
            new_token,
            past_key_values_data,
            current_length_data,
            self,
            hidden_state_new,
            sample_p
        )
        after_input = input_ids.clone()
        
        stats['total_steps'] += 1

        if is_llama3:
            if stop_token_id in input_ids[0, input_len:].tolist():
                break

        if self.tokenizer.eos_token_id in input_ids[0, input_len:].tolist():
            break
        if new_token > max_new_tokens:
            break
        if input_ids.shape[1] > max_length:
            break
    
    stats['total_tokens'] = new_token
    
    # Calculate summary statistics
    if stats['total_steps'] > 0:
        stats['avg_true_accept_length'] = sum(stats['true_accept_lengths']) / len(stats['true_accept_lengths'])
        stats['avg_predicted_accept_length'] = sum(stats['predicted_accept_lengths']) / len(stats['predicted_accept_lengths'])
        
        # Acceptance rate: (total accepted tokens) / (total candidates)
        max_candidates = candidates.shape[1] - 1 if 'candidates' in locals() else self.ea_layer.total_tokens
        stats['true_acceptance_rate'] = stats['avg_true_accept_length'] / max_candidates if max_candidates > 0 else 0
        stats['predicted_acceptance_rate'] = stats['avg_predicted_accept_length'] / max_candidates if max_candidates > 0 else 0
        stats['acceptance_rate_gap'] = abs(stats['predicted_acceptance_rate'] - stats['true_acceptance_rate'])
        
        # Early exit statistics
        stats['early_exit_rate'] = sum(stats['early_exit_triggered']) / len(stats['early_exit_triggered'])
        stats['avg_length_difference'] = sum([t - p for t, p in zip(stats['true_accept_lengths'], stats['predicted_accept_lengths'])]) / len(stats['true_accept_lengths'])
    else:
        stats['avg_true_accept_length'] = 0
        stats['avg_predicted_accept_length'] = 0
        stats['true_acceptance_rate'] = 0
        stats['predicted_acceptance_rate'] = 0
        stats['acceptance_rate_gap'] = 0
        stats['early_exit_rate'] = 0
        stats['avg_length_difference'] = 0
    
    if not log:
        return input_ids
    else:
        return input_ids, stats

EaModel.eagenerate_with_early_exit_hook = new_eagenerate_with_early_exit_hook


In [15]:
# Generate with early exit hook and collect statistics
output_ids, stats = model.eagenerate_with_early_exit_hook(
    input_ids=input_ids,
    log=True,  # Important: set log=True to get statistics
    max_new_tokens=1024,
    top_k=1,
    early_exit_hook=reject_all_exit_hook,
)

print(tokenizer.decode(output_ids[0], skip_special_tokens=True))
print(stats['total_steps'], len(output_ids[0]))

user
Complete the code I provided.from typing import List


def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """

assistant
<think>

</think>

Here's the completed code for your function:

```python
from typing import List

def filter_by_substring(strings: List[str], substring: str) -> List[str]:
    """ Filter an input list of strings only for ones that contain given substring
    >>> filter_by_substring([], 'a')
    []
    >>> filter_by_substring(['abc', 'bacd', 'cde', 'array'], 'a')
    ['abc', 'bacd', 'array']
    """
    return [s for s in strings if substring in s]
```

### Explanation:
- The function uses a list comprehension to filter the input list `strings`.
- It includes only those strings that contain the `substring` as a part 

In [40]:
# in each step, stats the average of top1 prob of next "accepted token length" tokens
neg_avg_top1_prob = 0.0
neg_cnt = 0
acc_avg_top1_prob = 0.0
acc_cnt = 0
for step in range(len(top1_probs)):
    accept_length = stats['true_accept_lengths'][step] + 1
    if accept_length == 1:
        # Find the next place where accept_length > 0
        for look_ahead in range(step+1, len(top1_probs)):
            if stats['true_accept_lengths'][look_ahead] > 0:
                accept_length = stats['true_accept_lengths'][look_ahead] + 1
                break
        neg_avg_top1_prob += sum(top1_probs[step:look_ahead])
        neg_cnt += look_ahead - step
    else:
        acc_avg_top1_prob += sum(top1_probs[step:step+accept_length])
        acc_cnt += accept_length + 1

print("Unaccepted token avg top1 prob:", neg_avg_top1_prob / neg_cnt)
print("Accepted tokens avg top1 prob for next accept_length tokens:", acc_avg_top1_prob / acc_cnt)

Unaccepted token avg top1 prob: 0.8999036153157552
Accepted tokens avg top1 prob for next accept_length tokens: tensor(0.7574, device='cuda:0')


In [28]:
import numpy as np

# ============================================================
# Utils
# ============================================================

def greedy_true_accept(true_accept_length):
    """
    方法一：greedy 使用 true_accept_length
    """
    N = len(true_accept_length)
    i = 0
    accepts = []
    path = []

    while i < N:
        L = int(true_accept_length[i])
        accepts.append(L)
        path.append((i, L))
        i = i + L + 1

    return np.mean(accepts), accepts, path


def oracle_top1_accept(true_accept_length, top1_prob, prob_threshold=0.9):
    """
    方法二：全知 oracle，根据连续 top1 prob 决定接受长度
    约束：L <= true_accept_length[i]
    策略：连续 token 的 min prob >= threshold
    """
    N = len(true_accept_length)
    i = 0
    accepts = []
    path = []

    while i < N:
        max_L = int(true_accept_length[i])
        L = 0
        for l in range(1, max_L + 1):
            if i + l < N and np.min(top1_prob[i+1:i+l+1]) >= prob_threshold:
                L = l
            else:
                break

        accepts.append(L)
        path.append((i, L))
        i = i + L + 1

    return np.mean(accepts), accepts, path


# ============================================================
# DP: Check if average >= lambda
# ============================================================

def dp_check_lambda(true_accept_length, lam):
    """
    判定是否存在一条路径，使得 average_accept_length >= lam
    返回 dp[0]
    """
    N = len(true_accept_length)
    dp = np.full(N + 1, -np.inf)
    dp[N] = 0.0

    for i in range(N - 1, -1, -1):
        best = -np.inf
        max_L = int(true_accept_length[i])
        for L in range(max_L + 1):
            j = i + L + 1
            if j <= N:
                val = (L - lam) + dp[j]
                best = max(best, val)
        dp[i] = best

    return dp[0]


def find_max_average_accept(true_accept_length, eps=1e-4):
    """
    二分搜索最大平均接受长度 lambda
    """
    lo, hi = 0.0, np.max(true_accept_length)

    while hi - lo > eps:
        mid = (lo + hi) / 2
        if dp_check_lambda(true_accept_length, mid) >= 0:
            lo = mid
        else:
            hi = mid

    return lo


def recover_optimal_path(true_accept_length, lam):
    """
    在给定最优 lambda 下恢复最优路径
    """
    N = len(true_accept_length)
    dp = np.full(N + 1, -np.inf)
    choice = np.zeros(N, dtype=int)
    dp[N] = 0.0

    for i in range(N - 1, -1, -1):
        best = -np.inf
        best_L = 0
        max_L = int(true_accept_length[i])
        for L in range(max_L + 1):
            j = i + L + 1
            if j <= N:
                val = (L - lam) + dp[j]
                if val > best:
                    best = val
                    best_L = L
        dp[i] = best
        choice[i] = best_L

    # rollout
    i = 0
    accepts = []
    path = []
    while i < N:
        L = choice[i]
        accepts.append(L)
        path.append((i, L))
        i = i + L + 1

    return np.mean(accepts), accepts, path


# ============================================================
# Example Test
# ============================================================

if __name__ == "__main__":
    # 模拟 true_accept_length（例如 MTP 验证结果）
    true_accept_length = np.array([x.cpu().numpy() for x in stats['true_accept_lengths']])

    # 模拟 top1_prob（带相关性）
    top1_prob = np.array(top1_probs)

    N = len(true_accept_length)

    print("========== Method 1: Greedy True Accept ==========")
    avg1, acc1, path1 = greedy_true_accept(true_accept_length)
    print(f"Average accept length: {avg1:.4f}")
    print(f"Steps: {len(acc1)}")

    print("\n========== Method 2: Oracle Top1 Prob ==========")
    avg2, acc2, path2 = oracle_top1_accept(
        true_accept_length,
        top1_prob,
        prob_threshold=0.9
    )
    print(f"Average accept length: {avg2:.4f}")
    print(f"Steps: {len(acc2)}")

    print("\n========== Method 3: Optimal DP (Upper Bound) ==========")
    lam_star = find_max_average_accept(true_accept_length)
    avg3, acc3, path3 = recover_optimal_path(true_accept_length, lam_star)

    print(f"Optimal average accept length (lambda*): {lam_star:.4f}")
    print(f"Recovered path average: {avg3:.4f}")
    print(f"Steps: {len(acc3)}")

    print("\n========== Summary ==========")
    print(f"Greedy true        : {avg1:.4f}")
    print(f"Oracle top1 prob   : {avg2:.4f}")
    print(f"DP optimal (upper) : {avg3:.4f}")


========== Method 1: Greedy True Accept ==========
Average accept length: 1.3733
Steps: 75

========== Method 2: Oracle Top1 Prob ==========
Average accept length: 0.9667
Steps: 90

========== Method 3: Optimal DP (Upper Bound) ==========
Optimal average accept length (lambda*): 1.3600
Recovered path average: 1.3600
Steps: 75

========== Summary ==========
Greedy true        : 1.3733
Oracle top1 prob   : 0.9667
DP optimal (upper) : 1.3600


In [22]:
import numpy as np

def find_true_optimal_average(true_accept_length):
    n = len(true_accept_length)
    # dist[i] 表示到达位置 i 时能获得的最大 (总长度 - rho * 总步数)
    # 这是一种 Dinkelbach 算法的思想，但为了直观，我们直接用搜索或改版的 DP
    
    # 既然我们要找的是最大平均值，我们可以尝试所有可能的步数
    # dp[i][s] 表示到达位置 i，恰好用了 s 步时的最大总接受长度
    dp = np.full((n + 1, n + 1), -1.0)
    parent = {} # 用于回溯路径: (i, s) -> (prev_i, prev_L)

    dp[0][0] = 0
    
    for s in range(n): # 步数
        for i in range(n): # 当前位置
            if dp[i][s] == -1: continue
            
            max_L = int(true_accept_length[i])
            # 尝试所有可能的接受长度 L (从 0 到 max_L)
            for L in range(max_L + 1):
                next_pos = i + L + 1
                if next_pos > n:
                    actual_L = n - i - 1
                    target_pos = n
                else:
                    actual_L = L
                    target_pos = next_pos
                
                new_len = dp[i][s] + actual_L
                if new_len > dp[target_pos][s+1]:
                    dp[target_pos][s+1] = new_len
                    parent[(target_pos, s+1)] = (i, actual_L)

    # 寻找全局最大平均值
    best_avg = -1
    best_config = (0, 0) # (pos, steps)

    for s in range(1, n + 1):
        avg = dp[n][s] / s
        if avg > best_avg:
            best_avg = avg
            best_config = (n, s)

    # 回溯路径
    path = []
    curr_pos, curr_s = best_config
    while curr_s > 0:
        prev_pos, L = parent[(curr_pos, curr_s)]
        path.append((prev_pos, L))
        curr_pos = prev_pos
        curr_s -= 1
    
    return best_avg, path[::-1]

# 测试逻辑
# true_accept_length = [0, 10, 0, 0, 0, 0, 0, 0, 0, 0, 0] # 故意在位置1放个大的
# avg, path = find_true_optimal_average(true_accept_length)
# 示例数据
true_accept_length = stats['true_accept_lengths']
avg, p = find_global_optimal_path(true_accept_length)
print(f"全局最优平均接受长度: {avg:.4f}")
print(p)

全局最优平均接受长度: 1.3600
[(0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (5, 0), (6, 0), (7, 0), (8, 1), (10, 0), (11, 1), (13, 4), (18, 3), (22, 2), (25, 0), (26, 2), (29, 3), (33, 0), (34, 0), (35, 1), (37, 6), (44, 7), (52, 4), (57, 2), (60, 3), (64, 4), (69, 3), (73, 2), (76, 3), (80, 2), (83, 3), (87, 3), (91, 2), (94, 4), (99, 1), (101, 2), (104, 0), (105, 2), (108, 0), (109, 0), (110, 3), (114, 3), (118, 0), (119, 1), (121, 1), (123, 0), (124, 1), (126, 0), (127, 0), (128, 1), (130, 4), (135, 0), (136, 2), (139, 0), (140, 0), (141, 1), (143, 1), (145, 0), (146, 1), (148, 4), (153, 0), (154, 2), (157, 1), (159, 0), (160, 0), (161, 0), (162, 0), (163, 0), (164, 1), (166, 1), (168, 2), (171, 0), (172, 1), (174, 1), (176, 0)]
